In [ ]:
from netCDF4 import Dataset
import os
import numpy as np
import matplotlib.pyplot as pl
import pandas as pd

In [ ]:
DATA_DIR = os.path.join("..", "data", "forcing")
#nc = Dataset(os.path.join(DATA_DIR, "HolVol_SAOD_-9500_1900_v1.0.nc"))

In [ ]:
#nc.close()

In [ ]:
# -9500 to 1900 ERF
nc = Dataset(os.path.join(DATA_DIR, "HolVol_SAOD_-9500_1900_v1.0.nc"))
aod550_mt = nc.variables['aod550'][:]
lat_mt = nc.variables['lat'][:]
time_mt = nc.variables['time'][:]
nc.close()
time_mt[-51*12]

# 1979 to 2018 ERF
nc = Dataset(os.path.join(DATA_DIR, "GloSSAC_V2.0.nc"))
data_glossac = nc.variables['Glossac_Aerosol_Optical_Depth'][:]
lat_glossac = nc.variables['lat'][:]
trp_hgt_glossac = nc.variables['trp_hgt'][:]  # lat, month
alt_glossac = nc.variables['alt'][:]
nc.close()
time_mt[-51*12]
lat_glossac
alt_glossac
data_glossac[0,0,:]

In [ ]:
lat_mt_bnds = np.concatenate([[90], 0.5*(lat_mt[1:]+lat_mt[:-1]), [-90]])
weights = -np.squeeze(np.diff(np.sin(np.radians(lat_mt_bnds))))

lat_glossac_bnds = np.concatenate(([-90], 0.5*(lat_glossac[1:]+lat_glossac[:-1]), [90]))
weights_glossac = np.diff(np.sin(np.radians(lat_glossac_bnds)))

aod_mt = np.zeros((len(time_mt)))
for i in range(len(time_mt)):
    aod_mt[i] = np.average(aod550_mt[i,:],weights=weights)

angstrom = (550/525)**(-2.33)

aod_glossac = np.zeros(480)
for i in range(480):
    aod_glossac[i] = np.average(data_glossac[i,:,2],weights=weights_glossac)*angstrom

In [ ]:
nc = Dataset(os.path.join(DATA_DIR, "CMIP_1850_2014_extinction_550nm_strat_only_v3.nc"))
ext = nc.variables['ext550'][:].transpose((2,1,0))  # time, height, lat
lev = nc.variables['altitude'][:]
lat = nc.variables['latitude'][:]
time = nc.variables['month'][:]
print(nc.variables['month'])
nc.close()

In [ ]:
lat_bnds = np.concatenate(([-90], 0.5*(lat[1:]+lat[:-1]), [90]))
weights = np.diff(np.sin(np.radians(lat_bnds)))
tax = np.zeros(165*12)
aod_cmip6 = np.zeros(165*12)
for i in range(0,1970,12):
    gl_mn_OD = np.average(np.sum(np.mean(ext[i:i+12,...],axis=0) * 0.5 ,axis=0),weights=weights) # 0.5 is thickness in km

for i in range(1980):
    aod_cmip6[i] = np.average(np.sum(ext[i,...] * 0.5,axis=0),weights=weights)

In [ ]:
aod = np.concatenate((aod_mt[:-51*12],aod_cmip6[:129*12],aod_glossac))
len(aod)

In [ ]:
aod[108000+28200:108000+28812] = (1-np.linspace(0,1,612))*aod_mt[-51*12:]+np.linspace(0,1,612)*aod_cmip6[:612]
aod[108000+29748:108000+29868] = (1-np.linspace(0,1,120))*aod_cmip6[1548:1668] + np.linspace(0,1,120)*aod_glossac[:120]
# repeat last year
aod = np.append(aod, aod[-12:])

In [ ]:
pl.plot(np.arange(1845+1/24,1901+1/24,1/12), aod_mt[(108000+28140):], label='Toohey & Sigl -500 to 1900 incl. background')
pl.plot(np.arange(1850+1/24,1905+1/24,1/12), aod_cmip6[:660], label='CMIP6')
pl.plot(np.arange(1845+1/24,1905+1/24,1/12), aod[(108000+28140):(108000+28860)], label='blended')
pl.legend()

In [ ]:
pl.plot(np.arange(1979+1/24,2019+1/24,1/12), aod_glossac, label='Glossac')
pl.plot(np.arange(1975+1/24,2015+1/24,1/12), aod_cmip6[-480:], label='CMIP6')
pl.plot(np.arange(1975+1/24,2020+1/24,1/12), aod[-540:], label='blended')
pl.legend()

In [ ]:
volc_erf_minus20 = -20 * (aod - np.mean(aod[:((9500+1750)*12)]))

In [ ]:
len(volc_erf_minus20)

In [ ]:
time = np.arange(-9500, 2020, 1/12)
len(time)

In [ ]:
df_out = pd.DataFrame({
    'year': time,
    'erf': volc_erf_minus20
})

In [ ]:
df_out.to_csv('../data/forcing/volcanic_ERF_monthly_-950001-201912.csv', index=False)